In [ ]:
from keras import models, layers, Input

In [ ]:
input_tensor = Input(shape=(64,))

x = layers.Dense(32, activation='relu')(input_tensor)

x = layers.Dense(32, activation='relu')(x)

output_tensor = layers.Dense(10, activation='softmax')(x)

#only now put the pieces together into a model leading from input to output
model = models.Model(inputs=input_tensor, outputs=output_tensor)

# compile and fit as usual

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accurcy'])

model.fit(x_train, y_train, epochs=10, batch_size=128)

### Multiple inputs example

Example 1: CNN to process image input, RNN to process text input
You work for an online store that sells clothing, like Etsy. You want to fit a model with the following specifications:
Inputs:
• a picture of a piece of clothing
• a text description of the clothing
Outputs:
• estimated sale price

We could use a network architecture like:
• a CNN to process the picture
• an RNN to process the text description
• combine activation output from the CNN and RNN
• dense output layer to predict price

Example 2: RNN to process two different text inputs
You work at a tech company, and are developing a user help page that will answer users’ questions based on help
documentation. You want to fit a model with the following specifications:
Inputs:
• a text question a user typed
• text of relevant help articles
Outputs:
• answer to the question, which is one of K categories (for example, K-1 possible solutions, or else send them to a
human being to answer the question.)
We could use a network architecture like:
• a first RNN to process the question
• a second RNN to process the text with the answers
• feed the output from the RNNs into a dense layer to generate a predicted class for the answer.

<img src="data/example.jpg">

In [3]:
from keras.models import Model
from keras import layers, Input

In [ ]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# first input layer is a text sequence.
# - We don't specify the length since it may differ across observations
# - data type is integer because we sparse-encoded the text
# - we can give the input layer a name if we want

text_input = Input(shape=(None,), dtype='int32', name='text')

# 64 dimensional word embedding of text input
embedding_text = layers.Embedding(64, text_vocabulary_size)(text_input)

# LSTM to process text
encoded_text = layers.LSTM(32)(embedding_text)

#**************

# second input layer is the question; same structure as for text input
question_input = Input(shape=(None, ), dtype='int32', name='question')

# 32 dimensional word embedding of question input
embedding_question = layers.Embedding(32, question_vocabulary_size)(question_input)

# LSTM to process question
encoded_question = layers.LSTM(16)(embedding_question)

# concatenate the activation outputs from the text LSTM and the question LSTM
# axis = -1 means to concatenate along the last axis.

concateneted = layers.concatenate([encoded_text, encoded_question], axis=-1)

# the answer is one of a fixed number of categories.
# probabilities for these categories are calculated using a softmax activation
# acting on the concatenated output from the question and the reference text
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concateneted)

# The network now has two inputs!!
model = Model(inputs=[text_input, question_input], outputs=answer)

# Compile as usual
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

# To fit, provide a dictionary with the values of the inputs
# names in the dictionary must match the names used when defining the layers
model.fit({'test': text, 'question': question}, answeres, epochs=10, batch_size=128)

You are kindof creepy (example from Chollet. . . ). You want to take a series of social media posts from a single anonymous
person and predict attributes of that person such as age, gender, and income level.
Input:
• social media posts
Output:
• age
• gender
• income level
Figure from Secton 7.1 of Chollet

In [4]:
from keras import layers
from keras import Input
from keras.models import Model

In [5]:
vocabulary_size = 50000
num_income_groups = 10

In [ ]:
# input layers
posts_input = Input(shape=(None,), dtype='int32', name='posts')

# word embedding and hidden layers
embedding_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedding_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='rleu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# separate output layers for each target variable
age_prediction = layers.Dense(1, name='age')(x)

incoming_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)

gender_predictoin = layers.Dense(1, activation='sigmoid', name='gender')(x)

# define model with list of outputs
model = Model(inputs=posts_input, outputs=[age_prediction, incoming_prediction, gender_predictoin])

# compile model, specifying dictionary of losses used for each output variable
# and their relative contributions to the final loss
model.compile(optimizer='rmsprop', loss={'age': 'mse',
                                         'income': 'categorical_crossentropy',
                                         'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1,
                            'gender': 10})

# fit model:
# x is posts
# y is dictionary of y s for each target variable
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_target}, epochs=10, batch_size=64)